In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

In [5]:
from google.colab import files
uploaded = files.upload()

Saving stock50.csv to stock50.csv


In [6]:
stock = pd.read_csv('stock50.csv')
stock.head()

,Stock_num,Date,Open,High,Min,Close,Stock_trade
0,50,20170103,67.69,67.88,67.51,67.88,2331
1,50,20170104,67.98,68.07,67.74,67.84,4284
2,50,20170105,67.84,68.45,67.79,68.45,4573
3,50,20170106,68.40,68.69,68.40,68.59,3752
4,50,20170109,68.69,68.87,68.26,68.40,5038


In [7]:
def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm
  
def train_windows(df, ref_day=5, predict_day=1):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day+1):
        X_train.append(np.array(df.iloc[i:i+ref_day,:-1]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]["y"]))
    return np.array(X_train), np.array(Y_train)

In [8]:
stock = stock.dropna()
stock['y'] = stock['Close'].shift(-1)
stock.iloc[:,2:4] = normalize(stock.iloc[:,2:4])
stock.head()

,Stock_num,Date,Open,High,Min,Close,Stock_trade,y
0,50,20170103,0.000000,0.000000,67.51,67.88,2331,67.84
1,50,20170104,0.015978,0.010383,67.74,67.84,4284,68.45
2,50,20170105,0.008264,0.031148,67.79,68.45,4573,68.59
3,50,20170106,0.039118,0.044262,68.40,68.59,3752,68.40
4,50,20170109,0.055096,0.054098,68.26,68.40,5038,68.35


In [9]:
stock['Date'] = pd.to_datetime(stock['Date'], format='%Y%m%d')
stock['weekday'] = stock['Date'].apply(lambda x: x.day)
stock['year'] = stock['Date'].apply(lambda x: x.year)
stock['Month'] = stock['Date'].apply(lambda x: x.month)
test = stock[-30:]
stock = stock[(stock['year']==2017)|((stock['year']==2018)&(stock['Month']<=3))|((stock['year']==2018)&(stock['Month']==4)&(stock['weekday']<=27))]
#stock & test
stock = stock.reset_index()
test = test.reset_index()
stock = stock.dropna()

In [10]:
test.head()

,index,Stock_num,Date,Open,High,Min,Close,Stock_trade,y,weekday,year,Month
0,292,50,2018-03-16,0.898623,0.891803,83.10,83.90,5510,83.85,16,2018,3
1,293,50,2018-03-19,0.879339,0.886339,83.45,83.85,2035,83.80,19,2018,3
2,294,50,2018-03-20,0.854545,0.869945,83.20,83.80,2895,83.85,20,2018,3
3,295,50,2018-03-21,0.887603,0.886339,83.60,83.85,4891,83.55,21,2018,3
4,296,50,2018-03-22,0.898623,0.913661,83.45,83.55,3429,82.10,22,2018,3


In [11]:
stock.drop(['Stock_num','High','Min','Open','Date','weekday','year','Month',"index"], axis=1, inplace=True)
test.drop(['Stock_num','Date','High','Min','Open','weekday','year','Month','index'], axis=1, inplace=True)
stock.head()

,Close,Stock_trade,y
0,67.88,2331,67.84
1,67.84,4284,68.45
2,68.45,4573,68.59
3,68.59,3752,68.40
4,68.40,5038,68.35


In [12]:
X_train, Y_train = train_windows(stock, 5, 1)
X_test, Y_test = train_windows(test, 5, 1)

In [13]:
class stock_lstm(keras.Model):
  def __init__(self,units):
      super(stock_lstm, self).__init__()
      

      self.rnn = Sequential([
          layers.SimpleRNN(units,input_shape=(5,2),dropout=0.2,return_sequences=True,unroll=True),
          layers.SimpleRNN(units,dropout=0.2,unroll=True)
      ])
      self.fc = layers.Dense(32)
      self.out = layers.Dense(1)
      


  def call(self, inputs, training=None):
    
      x = self.rnn(inputs)
      x = self.fc(x)
      out = self.out(x)

      return out

In [14]:
batchsz = 16
data = tf.data.Dataset.from_tensor_slices((X_train,Y_train))
data = data.batch(batchsz,drop_remainder=True)

data_test = tf.data.Dataset.from_tensor_slices((X_test,Y_test))
data_test = data_test.batch(batchsz,drop_remainder=True)

In [15]:
data_iter = iter(data)
samples = next(data_iter)
print(samples[0].shape,samples[1].shape)

(16, 5, 2) (16, 1)


In [16]:
units = 128
epochs = 10


rnn_model = stock_lstm(units)
rnn_model.compile(optimizer  =keras.optimizers.Adam(),loss="mean_absolute_error",metrics=['mean_absolute_error'])
rnn_model.fit(data,epochs=epochs, validation_data = data_test,shuffle=True)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

19/19 [==============================] - 0s 16ms/step - loss: 62.2732 - mean_absolute_error: 62.2732 - val_loss: 53.0556 - val_mean_absolute_error: 53.0556
Epoch 2/10
19/19 [==============================] - 0s 3ms/step - loss: 38.6257 - mean_absolute_error: 38.6257 - val_loss: 29.2978 - val_mean_absolute_error: 29.2978
Epoch 3/10
19/19 [==============================] - 0s 3ms/step - loss: 11.2019 - mean_absolute_error: 11.2019 - val_loss: 0.8318 - val_mean_absolute_error: 0.8318
Epoch 4/10
